# STA 220 Data & Web Technologies for Data Analysis

### Lecture 18, 3/4/25, SQL

### Announcements

- 

### Today's topics
- Structured Query Language

In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql

In [2]:
db = sql.connect("../data/suppliers.sqlite")

In [3]:
pd.read_sql("""
    SELECT * FROM parts
""", db)

,PartID,PartName,Color,Weight,City
0,1,Nut,Red,12.0,London
1,2,Bolt,Green,17.0,Paris
2,3,Screw,Blue,17.0,Oslo
3,4,Screw,Red,14.0,London
4,5,Cam,Blue,12.0,Paris
5,6,Cog,Red,19.0,London


In [5]:
pd.read_sql("""
    SELECT AVG(weight), city 
    FROM parts
    WHERE weight < 17
    GROUP BY city
    HAVING color IN ('Red', 'Blue')
""", db)

,AVG(weight),City
0,13.0,London
1,12.0,Paris


### Joins

A `JOIN` combines two tables using a column they have in common. `JOIN` is the SQL equivalent of Pandas' `.join()` method and `pd.merge()` function.

In [ ]:
pd.read_sql("SELECT * FROM supplierparts", db)

In [ ]:
pd.read_sql("""
    SELECT * FROM
    supplierparts
    LEFT JOIN
    parts
    ON supplierparts.partid = parts.partid
""", db)

SQLite supports two kinds of joins:
* Left join (`LEFT JOIN`): Keep all rows from the left table, merging rows from the right.
* Inner join (`INNER JOIN`): Keep only rows where there is a match.

Other DBMSes sometimes support two more kinds of joins:
* Right join: Keep all rows from the right table, merging rows from the left.
* Full join: Keep all rows from both tables, merging where there are matches.

In [ ]:
pd.read_sql("SELECT * FROM supplierparts", db)

In [ ]:
pd.read_sql("""
    SELECT * FROM
    supplierparts AS l
    LEFT JOIN
    suppliers AS r
    ON l.supplierid = r.supplierid;
""", db)

In [ ]:
pd.read_sql("""
    SELECT l.qty, r.* FROM
    supplierparts AS l
    LEFT JOIN
    suppliers AS r
    ON l.supplierid = r.supplierid;
""", db)

### Subqueries

You can write one or more `SELECT` queries within another:

In [ ]:
pd.read_sql("""
    SELECT * FROM
        (SELECT l.*, r.supplierid, r.qty FROM
        parts AS l
        INNER JOIN
        supplierparts AS r
        ON l.partid = r.partid) AS a
    INNER JOIN
        (SELECT * FROM suppliers
        WHERE status > 10) AS b
    ON a.supplierid = b.supplierid;
""", db)

In [ ]:
# You can paste strings into SQL queries, but make sure the string you paste in isn't
# compromised (e.g., user input)
val = pd.read_sql("SELECT AVG(weight) FROM parts", db).iloc[0, 0]
val

In [ ]:
query = "SELECT * FROM parts WHERE weight > " + format(val) + "; NEW QUERY"
query

In [ ]:
pd.read_sql(query, db) # pandas.read_sql allows only one statement at a time

### Database Operations: CRUD

So far we've only learned 1/4 of the basic operations most databases support. They are:

* Create (`CREATE`): Create a new table.
* Read (`SELECT`): Read some rows from a table.
* Update (`UPDATE`): Update rows in a table.
* Delete (`DROP`): Delete a table.

For data analysis, you'll usually only need to read data.

Once in a while, it can be useful to create tables to store complicated results. We have to go back to __sqlite3__ to run SQL commands other than `SELECT`.

In [ ]:
db.execute("""
CREATE TABLE newparts AS 
    SELECT *
    FROM parts
""")

In [ ]:
db.execute("""
CREATE TABLE adresses (
    SupplierID int,
    SupplierName varchar(255),
    Address varchar(255),
    Email varchar(255)
)
""")

# Force the database to run ("commit") the query.
# Otherwise the database evaluates queries lazily, so the table will not show up.
db.commit()

In [ ]:
pd.read_sql("SELECT * FROM adresses", db)

In [ ]:
pd.read_sql("SELECT * FROM sqlite_master", db)

In [ ]:
# append a new record
db.execute('''
INSERT INTO adresses (SupplierID, SupplierName, Address, Email)
VALUES (1, 'John Doe', 'nowhere', 'john@doe.com');
''')

In [ ]:
pd.read_sql("SELECT * FROM adresses", db)

In [ ]:
db.execute('''
INSERT INTO adresses (SupplierID, SupplierName, Email)
VALUES (2, 'Don Joe', 'don@joe.com');
''')

In [ ]:
db.execute("""
    UPDATE adresses
    SET Address='1 One Shields Avenue, Davis CA 95616'
    WHERE SupplierID=1
""")

In [ ]:
pd.read_sql("SELECT * FROM adresses", db)

Careful! `Update` changes all selected records. 

In [ ]:
# new example record
db.execute('''
INSERT INTO adresses (SupplierID, SupplierName, Email)
VALUES (1, 'Jane Doe', 'jane@doe.com');
''')

In [ ]:
pd.read_sql("SELECT * FROM adresses", db)

In [ ]:
# two records are updated
db.execute("""
    UPDATE adresses
    SET Address='Golden Gate Brg, San Francisco, CA'
    WHERE SupplierID=1
""")

In [ ]:
pd.read_sql("SELECT * FROM adresses", db)

In [ ]:
# two attrributes are set
db.execute("""
    UPDATE adresses
    SET SupplierID=3,
        Address='1 One Shields Avenue, Davis CA 95616'
    WHERE SupplierName='Jane Doe'
""")

In [ ]:
pd.read_sql("SELECT * FROM adresses", db)

In [ ]:
db.execute("""
UPDATE adresses
SET SupplierID=1
""")
pd.read_sql("SELECT * FROM adresses", db)

In [ ]:
db.execute("""
DELETE FROM adresses
WHERE SupplierName='Don Joe'
""")
pd.read_sql("SELECT * FROM adresses", db)

In [ ]:
db.execute("DROP TABLE newparts")
db.execute("DROP TABLE adresses")
#db.commit()

In [ ]:
db.close()

<img src="https://imgs.xkcd.com/comics/exploits_of_a_mom_2x.png">

## Other Databases

You can use the __sqlalchemy__ package to connect to many different kinds of databases (not only SQLite).

The `sqlalchemy.create_engine()` function creates an _engine_ to connect to a database. The `.raw_connection()` method on the returned engine opens a connection compatible with Pandas.

See the __sqlalchemy__ documentation for more info about [how to write a database URL](https://docs.sqlalchemy.org/en/latest/core/engines.html#database-urls).

In [ ]:
import sqlalchemy

engine = sqlalchemy.create_engine("sqlite:///../data/suppliers.sqlite")
conn = engine.raw_connection()

In [ ]:
conn

In [ ]:
pd.read_sql("SELECT * FROM sqlite_master", conn)

In [ ]:
engine.table_names()

In [ ]:
conn.close()

Unlike SQLite, most databases:

* Are not stored in a single file.
* Have a client-server design, where you connect to the database by logging in with a username and password.